In [17]:
import tensorflow as tf
import tensorflow_federated as tff

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [18]:
clients = [1, 2, 3]

In [19]:
import tensorflow as tf
import tensorflow_federated as tff

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Preprocess the data
x_train = x_train.astype(np.float32) / 255.0
x_test = x_test.astype(np.float32) / 255.0

# Create a TFF dataset from the training data
mnist_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
mnist_train = mnist_train.repeat(2).shuffle(5000).batch(100)

# Split the data for multiple clients
num_clients = 3
client_datasets = [
    mnist_train.shard(num_shards=num_clients, index=i) for i in range(num_clients)
]

# Create a federated dataset
federated_train_data = [
    client_dataset.map(lambda x, y: (tf.reshape(x, [-1]), y))
    for client_dataset in client_datasets
]

In [20]:
print(federated_train_data[0])

<_MapDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.uint8, name=None))>


In [21]:
import tensorflow as tf

model1 = tf.keras.Sequential(
    [
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(10, activation="softmax"),
    ]
)

model1.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)


model2 = tf.keras.Sequential(
    [
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(10, activation="softmax"),
    ]
)

model2.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

model3 = tf.keras.Sequential(
    [
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(10, activation="softmax"),
    ]
)

model3.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

In [23]:
import tensorflow as tf
import tensorflow_federated as tff


def model_fn():
    keras_model = tf.keras.Sequential(
        [
            tf.keras.layers.Flatten(input_shape=(28, 28)),
            tf.keras.layers.Dense(128, activation="relu"),
            tf.keras.layers.Dense(10, activation="softmax"),
        ]
    )
    return tff.learning.models.from_keras_model(keras_model)


def client_fn(model, dataset):
    model.compile(
        optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
    )
    model.fit(dataset, epochs=5)
    return model.get_weights()


def server_fn(model_weights):
    model = model_fn()
    model.set_weights(model_weights)
    return model


# Create a federated averaging algorithm
fed_avg = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.Adam(),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0),
)

# Build a federated learning process
federated_process = tff.learning.build_federated_averaging_process(
    model_fn, server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0)
)

# Train each model on a different client dataset
for i, client_dataset in enumerate(federated_train_data):
    print(f"Training model {i+1} on client dataset {i+1}")
    model_weights = client_fn(eval(f"model{i+1}"), client_dataset)
    server_model = server_fn(model_weights)
    eval(f"model{i+1}").set_weights(server_model.get_weights())

TypeError: from_keras_model() missing 2 required positional arguments: 'loss' and 'input_spec'